# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,roswell,34.0232,-84.3616,23.86,60,100,3.09,US,1713995255
1,1,edinburgh of the seven seas,-37.0676,-12.3116,16.10,73,100,6.27,SH,1713995255
2,2,bethel,41.3712,-73.4140,18.18,45,20,4.63,US,1713995256
3,3,papatowai,-46.5619,169.4708,15.35,60,22,4.50,NZ,1713995256
4,4,bhuj,23.2667,69.6667,24.22,76,0,4.88,IN,1713995248


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
City_by_Humid = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5
)

# Display the map
City_by_Humid

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Narrow down cities that fit criteria
comfy_cities_df = city_data_df[(city_data_df["Max Temp"] >= 10) & (city_data_df["Max Temp"] <= 26)]

# Display sample data
comfy_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,roswell,34.0232,-84.3616,23.86,60,100,3.09,US,1713995255
1,1,edinburgh of the seven seas,-37.0676,-12.3116,16.10,73,100,6.27,SH,1713995255
2,2,bethel,41.3712,-73.4140,18.18,45,20,4.63,US,1713995256
3,3,papatowai,-46.5619,169.4708,15.35,60,22,4.50,NZ,1713995256
4,4,bhuj,23.2667,69.6667,24.22,76,0,4.88,IN,1713995248
...,...,...,...,...,...,...,...,...,...,...
569,569,safi,32.1667,-8.8333,15.86,60,2,6.14,MA,1713995681
570,570,kapenguria,1.2511,35.0895,14.21,92,95,3.20,KE,1713995681
571,571,ipira,-12.1583,-39.7372,25.31,81,100,4.74,BR,1713995445
572,572,menongue,-14.6585,17.6910,17.38,43,86,2.35,AO,1713995682


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=comfy_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,roswell,US,34.0232,-84.3616,60,
1,edinburgh of the seven seas,SH,-37.0676,-12.3116,73,
2,bethel,US,41.3712,-73.4140,45,
3,papatowai,NZ,-46.5619,169.4708,60,
4,bhuj,IN,23.2667,69.6667,76,
...,...,...,...,...,...,...
569,safi,MA,32.1667,-8.8333,60,
570,kapenguria,KE,1.2511,35.0895,92,
571,ipira,BR,-12.1583,-39.7372,81,
572,menongue,AO,-14.6585,17.6910,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
import requests

# Set parameters to search for a hotel
limit = 1
radius = 100000
params = {
    "categories": "accommodation.hotel",
    "filter": f"circle:{hotel_df['Lng']},{hotel_df['Lat']},{radius}",
    "bias": f"proximity:{hotel_df['Lng']},{hotel_df['Lat']}",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Construct filter and bias parameters for the current city
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_name = "No hotel found"
    
    # Update the 'Hotel Name' column in the DataFrame
    hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_name}")

# Display sample data
print(hotel_df)

Starting hotel search
roswell - nearest hotel: Best Western Plus Hotel
edinburgh of the seven seas - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
papatowai - nearest hotel: Catlins Woodstock Lodge & Camp
bhuj - nearest hotel: Hotel Prince
port elizabeth - nearest hotel: Waterford Hotel
mil'kovo - nearest hotel: Дрозды
mansa - nearest hotel: Gurveer Grewal House
sarmiento - nearest hotel: Ismar
constantia - nearest hotel: Hotel Chérica
katabu - nearest hotel: Hotel Dina
whitehorse - nearest hotel: Town & Mountain Hotel
clairton - nearest hotel: Holiday Inn Express
sungai penuh - nearest hotel: Hotel Matahari
saint-pierre - nearest hotel: Tropic Hotel
santa maria - nearest hotel: Hotel Morotin
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
albany - nearest hotel: Tru
waitangi - nearest hotel: Hotel Chathams
boli - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
saldanha - nearest hotel: Hoedjesbaai
yokadouma - nearest hote

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:
# Create the map plot with customized hover tooltip
map_plot_final = hotel_df.hvplot.points(
    x="Lng",
    y="Lat", 
    hover_cols=["Country", "Hotel Name"],  # Combine the information for the hover tooltip
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot_final

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)